In [ ]:
pip install datasets

In [ ]:
from google.colab import drive
drive.mount('/content/drive')

In [ ]:
import datasets
from datasets import load_dataset, load_from_disk
import numpy as np
import pandas as pd
from datasets import Dataset

In [ ]:
datasets.logging.set_verbosity_error()
dataset = load_dataset("McAuley-Lab/Amazon-Reviews-2023", "raw_review_Books", trust_remote_code=True)

In [ ]:
from datetime import datetime

year = 2010

def filter_by_year(example):
    # Convert the timestamp from milliseconds to a datetime object
    dt = datetime.fromtimestamp(example['timestamp'] / 1000)
    return dt.year == year

# Apply the filter
filtered_dataset = dataset['full'].filter(filter_by_year)

filtered_dataset.save_to_disk("/content/drive/MyDrive/MIT805/Books_{}".format(year))

In [ ]:
datasets.logging.set_verbosity_error()
dataset_meta = load_from_disk("/content/drive/MyDrive/MIT805/Books_MetaData")
df_meta = dataset_meta.to_pandas()
df_meta.rename(columns={'title': 'book_title'}, inplace=True)

In [ ]:
for year in range(2010, 2024):
  datasets.logging.set_verbosity_error()
  dataset = load_from_disk("/content/drive/MyDrive/MIT805/Books_{}".format(year))
  df = dataset.to_pandas()

  merged_df = pd.merge(df, df_meta[['book_title', 'price', 'store', 'categories','parent_asin']], on='parent_asin')
  merged_df = merged_df[merged_df['price'] != "None"]

  hf_dataset = Dataset.from_pandas(merged_df)
  hf_dataset.save_to_disk("/content/drive/MyDrive/MIT805/Merged/Books_{}".format(year))
  print('Year {} is done!'.format(year))